In [5]:
import numpy
from numpy.random import randn

from openfermion.transforms._jordan_wigner import *

from openfermion.utils._trotter_exp_to_qgates import *

from openfermion.utils import (commutator, count_qubits, expectation,
                               hermitian_conjugated, normal_ordered,
                               jordan_wigner_sparse, jw_hartree_fock_state,
                               s_squared_operator, sz_operator)

from openfermion.utils._unitary_cc import (uccsd_generator,
                                           uccsd_singlet_generator,
                                           uccsd_singlet_get_packed_amplitudes,
                                           uccsd_singlet_paramsize)

In [6]:
test_orbitals = 3

single_amplitudes = randn(*(test_orbitals,) * 2)
double_amplitudes = randn(*(test_orbitals,) * 4)
print('singles')
print(single_amplitudes)
print('doubles')
print(double_amplitudes)


singles
[[-2.12979727  0.19601266 -0.19089402]
 [ 0.86309484  0.3745043   0.37282569]
 [-0.26461217  0.39227754 -0.5698435 ]]
doubles
[[[[-1.34287803e+00  1.68017839e-01  1.06546879e+00]
   [-1.97765834e-01  1.56925462e+00  8.43962442e-01]
   [-4.52994123e-01 -4.07297952e-01  5.64606100e-01]]

  [[ 2.74074693e-01  4.05285999e-01 -1.57790021e+00]
   [-7.74495626e-01 -6.54630556e-02 -2.14060670e-01]
   [ 4.38475287e-01  1.62109500e+00  9.46329135e-01]]

  [[-2.05506047e+00  4.72369842e-01 -4.09015507e-01]
   [ 6.93098721e-01 -1.34138198e+00 -5.88245088e-01]
   [-1.45202523e+00 -1.68890705e-01 -1.07660559e+00]]]


 [[[ 1.92432853e-01 -1.16032950e+00 -2.11369195e+00]
   [ 2.11817237e+00 -1.27600183e+00  2.35184601e+00]
   [-2.80175799e-01  3.76390044e-01 -2.67248714e-01]]

  [[-2.55619505e-01  5.77092253e-01 -6.35265325e-01]
   [ 2.03686079e+00  1.46166959e+00  8.92591240e-01]
   [-6.04446825e-01 -7.53213935e-01  4.35354377e-01]]

  [[ 1.50585141e+00  4.01231100e+00  3.12855627e-01]
   [ 1

In [7]:
generator = uccsd_generator(single_amplitudes, double_amplitudes)


qubit_op = jordan_wigner(generator)

#of_generator = trotter_operator_grouping(qubit_op)

# print('openfermion trotterized cluster operator')
# print(of_generator)
# print('')

qubit_op *= 1.0j

print('openferion pre trotterized generator')
print(generator)
print('')
print(qubit_op)

openferion pre trotterized generator
-0.024415013819882148 [0^ 0 0^ 1] +
0.18732341389708518 [0^ 0 0^ 2] +
-0.4718405270995154 [0^ 0 1^ 0] +
1.82487412038224 [0^ 0 1^ 1] +
0.1672557061763834 [0^ 0 1^ 2] +
1.6020663468581522 [0^ 0 2^ 0] +
-1.9131493581162426 [0^ 0 2^ 1] +
1.963715550906748 [0^ 0 2^ 2] +
-0.667082182875611 [0^ 1] +
0.4718405270995154 [0^ 1 0^ 0] +
-1.7128863681003974 [0^ 1 0^ 1] +
-1.141162928837435 [0^ 1 0^ 2] +
-2.102323843819901 [0^ 1 1^ 1] +
-0.18807701326792836 [0^ 1 1^ 2] +
-0.2546234335046131 [0^ 1 2^ 0] +
0.5009822652729545 [0^ 1 2^ 1] +
-0.2838911550863935 [0^ 1 2^ 2] +
0.07371814745293548 [0^ 2] +
-1.6020663468581522 [0^ 2 0^ 0] +
0.7525456411845289 [0^ 2 0^ 1] +
-1.1114880199765107 [0^ 2 0^ 2] +
0.2546234335046131 [0^ 2 1^ 0] +
-0.7369351525016719 [0^ 2 1^ 1] +
0.5122601571716504 [0^ 2 1^ 2] +
-0.09804024005469018 [0^ 2 2^ 1] +
-1.652885966933101 [0^ 2 2^ 2] +
0.667082182875611 [1^ 0] +
0.024415013819882148 [1^ 0 0^ 0] +
-1.993450707021767 [1^ 0 0^ 2] +
1.7128

In [8]:
from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
from openfermion.ops import QubitOperator


In [9]:
#check circuit as defined in pyquil
pauli_sum_representation = qubitop_to_pyquilpauli(qubit_op)
print(pauli_sum_representation)

(-1.4945342668408639+0j)*Y0*X1 + (1.4945342668408639+0j)*X0*Y1 + (-2.0660285949723+0j)*Y0*Z1*X2 + (2.0660285949723+0j)*X0*Z1*Y2 + (1.9816793592610096+0j)*Y1*X2 + (-1.9816793592610096+0j)*X1*Y2 + (-0.9548080844524449+0j)*Z0*Y1*X2 + (0.9548080844524449+0j)*Z0*X1*Y2 + (1.2760678855467957+0j)*Y0*X2 + (-1.2760678855467957+0j)*X0*Y2 + (0.04860362655582173+0j)*Y0*X1*Z2 + (-0.04860362655582173+0j)*X0*Y1*Z2


In [10]:
from pyquil.quil import Program
from forestopenfermion import exponentiate
from pyquil.gates import *

pyquil_program = exponentiate(qubit_op)
pyquil_program = Program(X(0), pyquil_program)
print(pyquil_program)
print(len(pyquil_program))

X 0
RX(pi/2) 0
H 1
CNOT 0 1
RZ(-2.9890685336817278) 1
CNOT 0 1
RX(-pi/2) 0
H 1
H 0
RX(pi/2) 1
CNOT 0 1
RZ(2.9890685336817278) 1
CNOT 0 1
H 0
RX(-pi/2) 1
RX(pi/2) 0
H 2
CNOT 0 1
CNOT 1 2
RZ(-4.1320571899446) 2
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 2
H 0
RX(pi/2) 2
CNOT 0 1
CNOT 1 2
RZ(4.1320571899446) 2
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 2
RX(pi/2) 1
H 2
CNOT 1 2
RZ(3.963358718522019) 2
CNOT 1 2
RX(-pi/2) 1
H 2
H 1
RX(pi/2) 2
CNOT 1 2
RZ(-3.963358718522019) 2
CNOT 1 2
H 1
RX(-pi/2) 2
RX(pi/2) 1
H 2
CNOT 0 1
CNOT 1 2
RZ(-1.9096161689048898) 2
CNOT 1 2
CNOT 0 1
RX(-pi/2) 1
H 2
H 1
RX(pi/2) 2
CNOT 0 1
CNOT 1 2
RZ(1.9096161689048898) 2
CNOT 1 2
CNOT 0 1
H 1
RX(-pi/2) 2
RX(pi/2) 0
H 2
CNOT 0 2
RZ(2.5521357710935915) 2
CNOT 0 2
RX(-pi/2) 0
H 2
H 0
RX(pi/2) 2
CNOT 0 2
RZ(-2.5521357710935915) 2
CNOT 0 2
H 0
RX(-pi/2) 2
RX(pi/2) 0
H 1
CNOT 0 1
CNOT 1 2
RZ(0.09720725311164347) 2
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 1
H 0
RX(pi/2) 1
CNOT 0 1
CNOT 1 2
RZ(-0.09720725311164347) 2
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 1


In [7]:
from pyquil.api import WavefunctionSimulator
from pyquil import Program

print('')
wfn = WavefunctionSimulator().wavefunction(pyquil_program)
print(wfn)


(-0.9601827064+0j)|01> + (-0.2793728159+0j)|10>


# The correct answer!

In [8]:
# (0.2671478502+0j)|01> + (0.9636555537+0j)|10>

In [9]:
#H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1  Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1

# Now begin importing trial circuit to Qforte

In [10]:
import qforte

In [11]:
#get from openferion
qforte_generator = qforte.build_from_openfermion(qubit_op)
qforte.smart_print(qforte_generator)


 Quantum operator:
(-1.4292258995086184+0j)[X0 Y1]
+ (1.4292258995086184+0j)[Y0 X1]


# Now try to exponentiate the full operator 

In [12]:
expn = qforte.trotterization.trotterize(qforte_generator)
qforte.smart_print(expn)
#expn.str()


 Quantum circuit:
[H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1  Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1]


In [13]:
qc2 = qforte.QuantumComputer(2)
qc2.apply_gate(qforte.make_gate('X',0,0))
#qc2.apply_gate(qforte.make_gate('X',1,1))
qc2.str()


['(0.000000 +0.000000 i) |00>',
 '(1.000000 +0.000000 i) |10>',
 '(0.000000 +0.000000 i) |01>',
 '(0.000000 +0.000000 i) |11>']

In [14]:
qc2.apply_circuit(expn)
qc2.str()

['(0.000000 +0.000000 i) |00>',
 '(-0.960183 +0.000000 i) |10>',
 '(-0.279373 +0.000000 i) |01>',
 '(0.000000 +0.000000 i) |11>']

In [15]:
#(0.3510953756+0j)|0011> + (-0.6045891379+0j)|0101> + (-0.1071633537+0j)|0110> + (0.1697168339+0j)|1001> + (-0.6312331758+0j)|1010> + (-0.2691856261+0j)|1100>

In [16]:
# Now try optemizing the operators... 